In [1]:
!pip install opencv-python mediapipe



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import time


In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils


In [4]:
def detect_gesture(landmarks):
    fingers = []

    # Thumb: tip (4), base (2)
    if landmarks[4].x < landmarks[3].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # Other fingers: tip > pip (open)
    finger_tips = [8, 12, 16, 20]
    finger_pips = [6, 10, 14, 18]
    
    for tip, pip in zip(finger_tips, finger_pips):
        if landmarks[tip].y < landmarks[pip].y:
            fingers.append(1)
        else:
            fingers.append(0)

    # Gesture 🤘🏻: index and pinky only up
    if fingers == [0, 1, 0, 0, 1]:
        return "🤘🏻"

    # Gesture 🫰🏻: thumb and index together (x distance small)
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    if abs(thumb_tip.x - index_tip.x) < 0.05 and abs(thumb_tip.y - index_tip.y) < 0.05:
        return "🫰🏻"
    
    return None


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the image for natural feel
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
            gesture = detect_gesture(handLms.landmark)

            if gesture == "🤘🏻":
                cv2.putText(frame, "Hi Viduni! How are you? I have something to tell you.", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
            elif gesture == "🫰🏻":
                cv2.putText(frame, "Something special I feel about you.", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,255), 2)

    cv2.imshow("Gesture Message", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
